<a href="https://colab.research.google.com/github/OmriGruman/object-detection/blob/master/yolov3_poker_cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [1]:
!git clone https://github.com/ultralytics/yolov3  # clone
%cd yolov3
%pip install -qr requirements.txt  # install

import torch
from yolov3 import utils
display = utils.notebook_init()  # checks

YOLOv3 🚀 v9.6.0-46-gca9a981 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.9/78.2 GB disk)


Some important parameters throughout the project

CURRENCY = https://universe.roboflow.com/ds/kCykUm7ZFj?key=Tt6fZrPf98

In [2]:
DATASET_LINK = "https://universe.roboflow.com/ds/GqTXeS669Z?key=a6UCBne4GF"

PROJECT_NAME = "poker-cards" 

#Data

Download dataset

In [3]:
%cd /content

!mkdir datasets
!curl -L {DATASET_LINK} > roboflow.zip
!unzip -q roboflow.zip -d /content/datasets/{PROJECT_NAME}
!rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   887  100   887    0     0   2142      0 --:--:-- --:--:-- --:--:--  2142
100 92.4M  100 92.4M    0     0  12.9M      0  0:00:07  0:00:07 --:--:-- 19.4M


Read classes

In [4]:
%cd /content/datasets/{PROJECT_NAME}/test

with open("_darknet.labels", 'r') as f:
    class_names = f.read().split('\n')
class_names

/content/datasets/poker-cards/test


['10 Diamonds',
 '10 Hearts',
 '10 Spades',
 '10 Trefoils',
 '2 Diamonds',
 '2 Hearts',
 '2 Spades',
 '2 Trefoils',
 '3 Diamonds',
 '3 Hearts',
 '3 Spades',
 '3 Trefoils',
 '4 Diamonds',
 '4 Hearts',
 '4 Spades',
 '4 Trefoils',
 '5 Diamonds',
 '5 Hearts',
 '5 Spades',
 '5 Trefoils',
 '59',
 '6 Diamonds',
 '6 Hearts',
 '6 Spades',
 '6 Trefoils',
 '7 Diamonds',
 '7 Hearts',
 '7 Spades',
 '7 Trefoils',
 '8 Diamonds',
 '8 Hearts',
 '8 Spades',
 '8 Trefoils',
 '9 Diamonds',
 '9 Hearts',
 '9 Spades',
 '9 Trefoils',
 'A Diamonds',
 'A Hearts',
 'A Spades',
 'A Trefoils',
 'J Diamonds',
 'J Hearts',
 'J Spades',
 'J Trefoils',
 'K Diamonds',
 'K Hearts',
 'K Spades',
 'K Trefoils',
 'Q Diamonds',
 'Q Hearts',
 'Q Spades',
 'Q Trefoils']

Prepare data

In [5]:
%cd /content/datasets/{PROJECT_NAME}

!mkdir images labels images/train images/valid images/test labels/train labels/valid labels/test
!mv train/*.jpg images/train
!mv valid/*.jpg images/valid
!mv test/*.jpg images/test
!mv train/*.txt labels/train
!mv valid/*.txt labels/valid
!mv test/*.txt labels/test
!rm -r train valid test

/content/datasets/poker-cards


Construct yaml file for the data

In [6]:
%cd /content/yolov3/data

import yaml

data_dir_path = f"../datasets/{PROJECT_NAME}"
train_path = "images/train"
valid_path = "images/valid"
test_path = "images/valid"

yaml_content = {
    'path': data_dir_path,
    'train': train_path,
    'val': valid_path,
    'test': test_path,
    'names': {
        i: name for i, name in enumerate(class_names)
    }
}

with open(f"{PROJECT_NAME}.yaml", 'w') as f:
    f.write(yaml.dump(yaml_content))

/content/yolov3/data


#Train

Train pretrained yolov3 model on custom data

In [ ]:
%cd /content/yolov3

# Run YOLOv3 on custom data
!python train.py --weights yolov3.pt --data {PROJECT_NAME}.yaml --name {PROJECT_NAME} --freeze 11 --patience 10 

/content/yolov3
train: weights=yolov3.pt, cfg=, data=poker-cards.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=poker-cards, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=10, freeze=[11], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 15274, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 15274 (delta 11), reused 24 (delta 6), pack-reused 15236
Receiving objects: 100% (15274/15274), 14.24 MiB | 11.94 MiB/s, done.
Command 'git fetch ultralytics' timed out after 5 seconds
YOLOv3 🚀 v9.6.0-46-gca9a981 Python-3.8.10 torch-

#Visualize

Plot some train results

Show results using Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train

#Evaluation

Detect bounding boxes on test set

In [ ]:
%cd /content/yolov3

!python detect.py --weights runs/train/{PROJECT_NAME}/weights/best.pt --img 640 --conf 0.6 --source /content/datasets/{PROJECT_NAME}/images/test --name {PROJECT_NAME}

In [ ]:
%cd /content/yolov3/runs/detect/{PROJECT_NAME}

from os import listdir
import matplotlib.pyplot as plt
from random import sample

SIZE = 5
_, axarr = plt.subplots(SIZE, SIZE, figsize=(25, 25))

image_files = listdir()

images = iter(sample([plt.imread(image_name) for image_name in image_files], SIZE ** 2))

for r in range(SIZE):
    for c in range(SIZE):
        axarr[r, c].imshow(next(images))

#Video

Try to detect sign language in video

In [ ]:
%cd /content/yolov3

!pip install pafy
!pip install youtube_dl==2020.12.2

#for c in range(40, 70, 5): 
!python detect.py --weights /content/yolov3/runs/train/{PROJECT_NAME}/weights/last.pt --img 640 --conf 0.25 --source 'https://www.youtube.com/watch?v=A6hNKlqmlFk' --name {PROJECT_NAME}